# Simple grokking demo with superposition

In [3]:
# Import stuff
import numpy as np
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import einops

## Generate Dataset

In [6]:
# Setup
p = 97
train_frac  = 0.3
device = "cpu"

In [7]:
a_vec = einops.repeat(t.arange(p),"i -> (i j)",j=p)
b_vec = einops.repeat(t.arange(p),"j -> (i j)",i=p)
eq_vec = einops.repeat(t.tensor(p)," -> i",i= p**2)
# The dataset consists of pairs (x,y) with x = (a,b,eq) and y = a+b mod p
# we randomly permute the dataset and split it into train and test dataset
dataset = t.stack([a_vec,b_vec,eq_vec],dim=1).to(device=device)
labels = (dataset[:,0] + dataset[:,1]) % p
indices = t.randperm(p**2)
train_indices = indices[:int(train_frac*p**2)]
test_indices = indices[int(train_frac*p**2):]

train_dataset = dataset[train_indices]
train_labels = labels[train_indices]

test_dataset = dataset[test_indices]
test_labels = labels[test_indices]
print(train_dataset)
print(train_labels)

tensor([[71, 77, 97],
        [91,  9, 97],
        [33, 73, 97],
        ...,
        [ 6, 51, 97],
        [13, 39, 97],
        [75, 27, 97]])
tensor([51,  3,  9,  ..., 57, 52,  5])


## Create a simple one layer transformer model

This is sort of the easiest possible transformer model. It takes as input the one hot encoded token sequence (a,b,=), where = is set to p for convenience. The architecture is:

0. Token: the tokens $t_0,t_1,t_2$ are one hot encoded d_vocab dimensional vectors and the input sequence is $$t = (t_0,t_1,t_2)^T$$
1. Embedding: The tokens are embedded in the d_model dimensional space by a learnable matrix W_E $$x_0 = Embed(t) = t @W_E$$
2. Positional Embedding: The positional Embedding is implemented by adding a d_model dimensional vector to each embedded vector, depending on the token position. It is also learned. $$x_1 = x_0 + W_{pos}$$
3. Attention layer: A simple attention layer with num_heads = 4 attention heads
$$x_2 = x_1 + Attention(x_1)$$
4. MLP layer: A simple mlp with one hidden layer with ReLU activation function and no bias:
$$x_3 = x_2 + MLP(x_2)$$
5. Unembedding: a learned unembedding matrix W_U that maps back to the vocab.
$$x_4 = x_3 @ W_U$$

In [8]:
#Config for the transformer architecture
d_vocab = p+1   #The input consists of the numbers from 0 to p-1 and p for the equal sign
d_model = 128   #dimension of the model
n_ctx = 3       #context length (a,b,=) where = is encoded as 97
num_heads = 4
d_heads = d_model//num_heads
d_mlp = 4 * d_model
act_type = "ReLU"

In [4]:
from models.one_layer_transformer import Transformer

In [9]:
model = Transformer(d_model,d_mlp,d_heads,d_vocab,num_heads,n_ctx,act_type).to(device=device)


## Training

In [15]:
from torch.utils.data import DataLoader, TensorDataset

import tqdm.auto as tqdm

/opt/anaconda3/envs/ncl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#training parameters
n_epoch = 30000
lr = 1e-3
wd = 1.
DATA_SEED = 346
betas = (0.9,0.98)

In [12]:
# Define optimizer
optimizer = t.optim.AdamW(model.parameters(),lr=lr, betas=betas,weight_decay=wd)

In [13]:
#Define cross entropy loss
def loss_fn(logits,labels):
    if len(logits.shape) == 3:
        logits = logits[:,-1]
    logits = logits.to(t.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1,index = labels[:,None])[:,0]
    return -correct_log_probs.mean()

In [ ]:
train_losses = []
test_losses = []
for epoch in tqdm.tqdm(range(n_epoch)):
    train_logits = model(train_dataset)
    train_loss = loss_fn(train_logits,train_labels)
    train_loss.backward()
    train_losses.append(train_loss.item())
    optimizer.step()
    optimizer.zero_grad()
    with t.inference_mode():
        test_logits = model(test_dataset)
        test_loss = loss_fn(test_logits,test_labels)
        test_losses.append(test_loss.item())

    if ((epoch+1)%100)==0:
        print(f"Epoch {epoch} Train Loss {train_loss.item()} Test loss {test_loss.item()}")

  0%|          | 102/30000 [00:09<45:53, 10.86it/s]

Epoch 99 Train Loss 1.2547569312723175 Test loss 8.574755202910724


  1%|          | 202/30000 [00:18<43:30, 11.41it/s]

Epoch 199 Train Loss 0.013822463495749922 Test loss 14.162979331560935


  1%|          | 302/30000 [00:27<43:19, 11.43it/s]

...

  6%|▋         | 1901/30000 [02:56<46:10, 10.14it/s]

Epoch 1899 Train Loss 2.714062681167369e-07 Test loss 19.929136759903816


  7%|▋         | 2002/30000 [03:06<43:50, 10.64it/s]

Epoch 1999 Train Loss 2.6985457253635636e-07 Test loss 19.616734280234798


  7%|▋         | 2030/30000 [03:08<39:49, 11.71it/s]

## Plot results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses,label="Training Loss")
plt.plot(test_losses,label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Log Loss")
plt.yscale("log")
plt.title("Traing and Test loss")
plt.legend()
plt.show()